# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [29]:
city_weather_path = "weather_data/city_weather.csv"
city_weather_df = pd.read_csv(city_weather_path)
city_weather_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint George,37.1041,-113.5841,82.00,8,1,12.66,US,1618872464
1,Rikitea,-23.1203,-134.9692,78.13,73,0,10.56,PF,1618872197
2,Mataura,-46.1927,168.8643,62.01,62,0,5.01,NZ,1618872198
3,Balabac,7.9868,117.0645,81.05,76,95,15.79,PH,1618872198
4,Bredasdorp,-34.5322,20.0403,60.80,67,95,19.57,ZA,1618872198
...,...,...,...,...,...,...,...,...,...
564,Goderich,43.7501,-81.7165,44.60,87,100,16.11,CA,1618872366
565,Watsa,3.0372,29.5355,65.01,87,99,3.67,CD,1618872366
566,Coyhaique,-45.5752,-72.0662,48.20,71,75,4.61,CL,1618872367
567,Winnemucca,40.9730,-117.7357,71.60,12,1,14.97,US,1618872367


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
#adding in gmaps
gmaps.configure(api_key=g_key)
#adding cities locations as latitude and longitude
cities = city_weather_df[["Lat","Long"]]
#setting up the humidity as a number
humidity = city_weather_df["Humidity"].astype(float)

In [31]:
#creating heat map
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(cities, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
